In [1]:
!wget ftp://ftp.ncbi.nlm.nih.gov/genomes/all/GCA/000/001/405/GCA_000001405.29_GRCh38.p14/GCA_000001405.29_GRCh38.p14_genomic.fna.gz

'wget' is not recognized as an internal or external command,
operable program or batch file.


In [ ]:
import gzip
import shutil

with gzip.open('GCA_000001405.29_GRCh38.p14_genomic.fna.gz', 'rb') as f_in:
    with open('GCA_000001405.29_GRCh38.p14_genomic.fna', 'wb') as f_out:
        shutil.copyfileobj(f_in, f_out)

In [2]:
!makeblastdb -in GCA_000001405.29_GRCh38.p14_genomic.fna -dbtype nucl -out human_genome_db

'makeblastdb' is not recognized as an internal or external command,
operable program or batch file.


In [ ]:
from Bio.Blast.Applications import NcbiblastnCommandline
from Bio import SeqIO

# Load sequences from the sample_2.fasta file
sequences = list(SeqIO.parse("sample_2.fasta", "fasta"))

# Assume the first sequence is human and the rest are Toxoplasma gondii
human_sequence = str(sequences[0].seq)
toxoplasma_sequences = [seq for seq in sequences[1:]]

# Save human sequence to a file to use as subject in BLAST
with open("human_sequence.fasta", "w") as human_file:
    human_file.write(">human_sequence\n")
    human_file.write(human_sequence)

# Perform BLAST search for each Toxoplasma gondii sequence against the human genome and save results to files
for idx, seq_record in enumerate(toxoplasma_sequences):
    print(f"Performing BLAST search for sequence {idx+1}...")
    toxo_filename = f"toxo_sequence_{idx+1}.fasta"
    with open(toxo_filename, "w") as toxo_file:
        toxo_file.write(f">{seq_record.id}\n")
        toxo_file.write(str(seq_record.seq))
    
    # Run local BLAST search using the command line
    output_filename = f"blast_result_{idx+1}.xml"
    blastn_cline = NcbiblastnCommandline(query=toxo_filename, db="human_genome_db", evalue=0.001, outfmt=5, out=output_filename)
    stdout, stderr = blastn_cline()
    print(f"Results saved to {output_filename}")

print("BLAST searches completed.")